# Automatic Trial Videos Extraction (ReachMaster Data - HDF File)
### Authors: Harnoor Dhingra, Brett Nelson (NSDS Lab, LBNL)
#### Inputs: ReachMaster's HDF file and Analysis_Utils.py (in the same directory).
#### Outputs: A Directory of the format 'Video-Extraction-RM[Rat Number]-[Date]-[Session Number]' with Sub-directories 'Successful_Trials' and 'Trials.' These include all the trial video obtained from hours long experiment videos.

In [187]:
import numpy as np
import seaborn as sns
import pandas as pd
import pdb
import matplotlib.pyplot as plt
import seaborn as sns
from Analysis_Utils import preprocessing_df as preprocessing
from Analysis_Utils import query_df
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import cv2
from sys import exit
import glob

# Importing Data

In [188]:
def import_robot_data(df_path):
    df = pd.read_hdf(df_path)
    df = preprocessing(df)
    return df
import ast

In [189]:
hdf = import_robot_data('/Users/harnoor_dhingra/Desktop/Project Thesis/Project - Robot Data_hdf file_19Aug/default_save.h5')
hdf = hdf.reset_index(drop=True)

In [190]:
def update_lines(num, dataLines, lines) :
    for line, data in zip(lines, dataLines) :
        line.set_data(data[0:2, 0:num])
        line.set_3d_properties(data[2,0:num])
    return lines

In [191]:
hdf.head()

,rat,S,Date,dim,time,m_start,m_stop,SF,t_m,lick,...,x_p,y_p,z_p,x0,y0,z0,moving,RW,r_start,r_stop
0,RM9,S2,18,3,"[20.859866697184316, 20.8858667200725, 20.9028...","[31.668866712443105, 67.45086677729296, 105.33...","[35.674866754404775, 71.4538667982738, 109.495...","[2, 3, 4, 5, 6, 7, 8, 9, 13, 17, 19, 20, 21, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 35.6913, 35.6982, 36.564233333333334, 10...",...,655.4 916.8 916.8 655.4 655.4 655.4 916.8 655....,794.8 648.8 648.8 449.0 794.8 794.8 648.8 794....,361.6 571.1 571.1 361.6 707.5 707.5 571.1 707....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[1513, 6563, 12020, 13242, 20341, 21483, 22472...","[2077, 7124, 12612, 13736, 21060, 22046, 23017..."
1,RM9,S1,18,3,"[20.414766451136302, 20.440766474024485, 20.48...","[89.86176647783918, 99.30376662851972, 174.025...","[93.86576666475935, 103.30476655604048, 179.71...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 179.05716666666666, 179.09396666666666, ...",...,614.4 614.4 614.4 614.4 614.4 614.4 614.4 614....,608.0 608.0 608.0 608.0 608.0 608.0 608.0 608....,531.0 531.0 531.0 531.0 531.0 531.0 531.0 531....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[9864, 11201, 21864, 24920, 26074, 27904, 2992...","[10430, 11766, 22667, 25452, 26535, 28404, 304..."
2,RM9,S3,18,3,"[19.026683361943583, 19.089683564122538, 19.10...","[34.658683569844584, 117.32368353169761, 142.7...","[38.665683539327006, 121.32568338673912, 146.1...","[2, 3, 6, 9, 12, 14, 15, 16, 18, 20, 21, 23]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 122.51423333333334, 122.51763333333334, ...",...,655.4 916.8 916.8 655.4 655.4 655.4 916.8 655....,794.8 648.8 648.8 449.0 794.8 794.8 648.8 794....,361.6 571.1 571.1 361.6 707.5 707.5 571.1 707....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[2191, 14018, 17703, 18863, 21270, 31546, 3251...","[2753, 14597, 18205, 19352, 21850, 32097, 3301..."
3,RM9,S4,18,3,"[21.447966702906342, 21.479966767756196, 21.52...","[34.8089667639415, 152.94096673342392, 159.957...","[38.812966712443085, 156.9469667753856, 163.96...","[4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 19, 20, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 214.57526666666666, 214.68453333333332, ...",...,655.4 916.8 916.8 655.4 655.4 655.4 916.8 655....,794.8 648.8 648.8 449.0 794.8 794.8 648.8 794....,361.6 571.1 571.1 361.6 707.5 707.5 571.1 707....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[1873, 18701, 19694, 20832, 26812, 32196, 3331...","[2439, 19269, 20261, 21394, 27761, 32686, 3388..."
4,RM9,S1,19,3,"[20.34046666857398, 20.412466695276862, 20.439...","[36.99546687838233, 93.67546670672095, 103.992...","[40.99946682688392, 97.68146674868262, 107.997...","[3, 5, 9, 12, 13, 16, 19, 20, 21, 22, 23, 27, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 108.04173333333334, 108.04273333333333, ...",...,655.4 916.8 916.8 655.4 655.4 655.4 916.8 655....,794.8 648.8 648.8 449.0 794.8 794.8 648.8 794....,361.6 571.1 571.1 361.6 707.5 707.5 571.1 707....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 ...,"[2332, 10392, 11884, 13810, 15792, 18687, 2008...","[2898, 10977, 12461, 14321, 16370, 19248, 2064..."


In [228]:
# for col in hdf.columns: 
#     print(col) 

In [229]:
# hdf.index

# Indexing as Rat - Date - Session Number

In [194]:
# Sorting and Multi-Indexing
df = hdf.sort_values(['rat', 'Date', 'S'])
df = df.set_index(['rat', 'Date', 'S'])
df.head()

dim                                               time  \
rat  Date S                                                           
RM10 17   S1   3  [20.343600175476013, 20.411600015258728, 20.43...   
          S2   3  [20.37000005340576, 20.426000047683715, 20.440...   
          S3   3  [19.484500066757164, 19.50050009918209, 19.525...   
     18   S1   3  [20.1379999485016, 20.193000015258804, 20.2290...   
          S2   3  [21.12278335431415, 21.14078348019916, 21.2137...   

                                                        m_start  \
rat  Date S                                                       
RM10 17   S1  [142.29960002670282, 288.8056000686645, 298.08...   
          S2  [274.2350000629425, 285.5069998989105, 302.520...   
          S3  [61.913499967575035, 188.4325000209808, 202.27...   
     18   S1  [157.60700004959108, 190.90199988746645, 226.9...   
          S2  [36.928783348592106, 44.06178348401386, 93.821...   

                                                         m_stop  \
rat  Date S                                                       
RM10 17   S1  [146.30460014114374, 292.61860003242487, 302.6...   
          S2  [277.86499993896484, 292.25099985694885, 305.7...   
          S3  [65.9195000095367, 191.8205000324249, 205.4205...   
     18   S1  [161.615999874115, 194.55699985885622, 230.535...   
          S2  [40.93078344205219, 48.06778328755695, 97.8277...   

                                                           SF  \
rat  Date S                                                     
RM10 17   S1                                           [3, 4]   
          S2                                  [1, 2, 3, 5, 6]   
          S3  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]   
     18   S1      [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15]   
          S2                   [7, 9, 15, 20, 21, 22, 23, 24]   

                                                            t_m  \
rat  Date S                                                       
RM10 17   S1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
          S2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
          S3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
     18   S1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
          S2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                           lick  \
rat  Date S                                                       
RM10 17   S1  [0.0, 541.2396333333334, 541.2472666666666, 54...   
          S2  [0.0, 279.68016666666665, 279.7112666666667, 2...   
          S3  [0.0, 121.46453333333334, 190.52003333333334, ...   
     18   S1  [0.0, 82.6542, 82.6982, 83.1201, 194.108833333...   
          S2  [0.0, 209.08166666666668, 209.17086666666665, ...   

                                                            r_x  \
rat  Date S                                                       
RM10 17   S1  [3.041215709448993, 3.000524363480997, 5.01705...   
          S2  [5.005960357380994, 5.01335878392063, 4.758113...   
          S3  [5.01335878392063, 5.01335878392063, 4.7581130...   
     18   S1  [5.01335878392063, 4.857991826588281, 4.243922...   
          S2  [1.7608255164332929, 2.834059766339194, 2.7452...   

                                                            r_y  \
rat  Date S                                                       
RM10 17   S1  [2.9598330175130005, 2.9561338042431826, 2.985...   
          S2  [2.982028297131908, 2.9672314440526364, 2.9709...   
          S3  [2.9968251502111793, 2.9968251502111793, 2.974...   
     18   S1  [2.989426723671543, 2.9857275104017256, 2.9931...   
          S2  [2.3975526005006884, 3.21877794640025, 3.41853...   

                                                            r_z  \
rat  Date S                                                       
RM10 17   S1  [2.619505396689759, 2.6158061834199415, 2.0979...   
          S2  [2.2902754156759713, 2.2

In [227]:
# df.index

# Adding Number of Successful Trials Column

In [196]:
# Appending the number of Successful Trials column
df_1 = df['SF'].apply(pd.Series)
df_1 = df_1.rename(columns = lambda x : 'SF_' + str(x))
df_numSF_t = df_1.T
df_numSF_t.loc['Number of successful trials'] = df_numSF_t.count()
df_numSF = df_numSF_t.T
df['NSF'] = df_numSF['Number of successful trials'].astype(int)
# df.head()

# Adding Total Trials Column

In [197]:
# Appending the number of Total trials
df_r_start_1 = df['r_start'].apply(pd.Series)
df_r_start_1 = df_r_start_1.rename(columns = lambda x : 'r_start' + str(x))
df_r_start_t = df_r_start_1.T
df_r_start_t.loc['Total trials'] = df_r_start_t.count()
df_r_start = df_r_start_t.T
df['Total Trials'] = df_r_start['Total trials'].astype(int)
df

dim                                               time  \
rat  Date S                                                           
RM10 17   S1   3  [20.343600175476013, 20.411600015258728, 20.43...   
          S2   3  [20.37000005340576, 20.426000047683715, 20.440...   
          S3   3  [19.484500066757164, 19.50050009918209, 19.525...   
     18   S1   3  [20.1379999485016, 20.193000015258804, 20.2290...   
          S2   3  [21.12278335431415, 21.14078348019916, 21.2137...   
          S3   3  [21.175866720072463, 21.219866630427077, 21.23...   
          S4   3  [20.174799883651758, 20.243799889373804, 20.27...   
     19   S1   3  [20.756849780654875, 20.79084993896481, 20.813...   
          S2   3  [20.03881668574013, 20.069816584650653, 20.124...   
          S3   3  [19.729200024795546, 19.768199820709242, 19.79...   
          S4   3  [21.175533432515522, 21.23353328183498, 21.250...   
     20   S1   3  [20.31741670672102, 20.346416750590038, 20.364...   
          S2   3  [19.442916533152243, 19.463916680018087, 19.48...   
          S3   3  [19.555566632334376, 19.58856662470498, 19.662...   
          S4   3  [20.028816741053333, 20.05081657702135, 20.125...   
     25   S1   3  [20.94116665522256, 20.962166802088404, 20.982...   
          S2   3  [20.561699826431322, 20.617699820709277, 20.64...   
          S3   3  [19.203366571299284, 19.274366670481413, 19.29...   
          S4   3  [19.072666783014938, 19.106666702906296, 19.17...   
     26   S1   3  [21.39170013923649, 21.416699996185343, 21.449...   
          S2   3  [19.747466874567635, 19.800466847864755, 19.82...   
          S3   3  [19.367233312352482, 19.385233199818913, 19.43...   
          S4   3  [18.82786671053566, 18.844866670481338, 18.862...   
     27   S1   3  [19.348833358128786, 19.376833474477053, 19.39...   
          S2   3  [19.390599961853013, 19.408600087738023, 19.45...   
          S3   3  [19.48921666094475, 19.505216693369675, 19.517...   
          S4   3  [19.418633346684828, 19.434633379109755, 19.47...   
     28   S1   3  [20.67056670099896, 20.702566765848815, 20.744...   
          S2   3  [19.875133398183152, 19.89113343060808, 19.905...   
          S3   3  [19.365666624705, 19.38266658465068, 19.402666...   
...           ..                                                ...   
RM9  17   S1   3  [20.08476657129927, 20.136766617075637, 20.152...   
          S2   3  [21.31263349736537, 21.37263344014491, 21.3986...   
          S3   3  [21.278200051498402, 21.29420008392333, 21.335...   
     18   S1   3  [20.414766451136302, 20.440766474024485, 20.48...   
          S2   3  [20.859866697184316, 20.8858667200725, 20.9028...   
          S3   3  [19.026683361943583, 19.089683564122538, 19.10...   
          S4   3  [21.447966702906342, 21.479966767756196, 21.52...   
     19   S1   3  [20.34046666857398, 20.412466695276862, 20.439...   
          S2   3  [20.732933211263003, 20.769933390553774, 20.82...   
          S3   3  [20.75843331044507, 20.785433260854006, 20.860...   
          S4   3  [20.39856660563146, 20.430566670481312, 20.494...   
     20   S1   3  [19.414733276112884, 19.43573342297873, 19.511...   
          S2   3  [21.781683314259908, 21.81368337910976, 21.888...   
          S3   3  [20.540516552225768, 20.60651653696698, 20.623...   
          S4   3  [20.42229991035458, 20.456300068664518, 20.477...   
     25   S1   3  [19.758366592280026, 19.835366733423825, 19.85...   
          S2   3  [-464.9342998874664, -464.8682999027252, -464....   
          S3   3  [20.871133501179997, 20.936133319981877, 20.95...   
          S4   3  [20.69669991989133, 20.74369985122678, 20.7686...   
     26   S1   3  [19.548800186920175, 19.578800158309946, 19.61...   
          S2   3  [20.16585012969972, 20.18385001716615, 20.2088...   
          S3   3  [19.343699908447206, 19.360699868392885, 19.38...   
          S4   3  [19.247033346684702, 19.262033451588877, 19.27...   
     27   S1   3  [20.162333226521696

# Multi-Index Data Extraction Examples

In [198]:
# Sample-1 Indexing based on Rat name, Session and Date
df.loc['RM10']

dim                                               time  \
Date S                                                           
17   S1   3  [20.343600175476013, 20.411600015258728, 20.43...   
     S2   3  [20.37000005340576, 20.426000047683715, 20.440...   
     S3   3  [19.484500066757164, 19.50050009918209, 19.525...   
18   S1   3  [20.1379999485016, 20.193000015258804, 20.2290...   
     S2   3  [21.12278335431415, 21.14078348019916, 21.2137...   
     S3   3  [21.175866720072463, 21.219866630427077, 21.23...   
     S4   3  [20.174799883651758, 20.243799889373804, 20.27...   
19   S1   3  [20.756849780654875, 20.79084993896481, 20.813...   
     S2   3  [20.03881668574013, 20.069816584650653, 20.124...   
     S3   3  [19.729200024795546, 19.768199820709242, 19.79...   
     S4   3  [21.175533432515522, 21.23353328183498, 21.250...   
20   S1   3  [20.31741670672102, 20.346416750590038, 20.364...   
     S2   3  [19.442916533152243, 19.463916680018087, 19.48...   
     S3   3  [19.555566632334376, 19.58856662470498, 19.662...   
     S4   3  [20.028816741053333, 20.05081657702135, 20.125...   
25   S1   3  [20.94116665522256, 20.962166802088404, 20.982...   
     S2   3  [20.561699826431322, 20.617699820709277, 20.64...   
     S3   3  [19.203366571299284, 19.274366670481413, 19.29...   
     S4   3  [19.072666783014938, 19.106666702906296, 19.17...   
26   S1   3  [21.39170013923649, 21.416699996185343, 21.449...   
     S2   3  [19.747466874567635, 19.800466847864755, 19.82...   
     S3   3  [19.367233312352482, 19.385233199818913, 19.43...   
     S4   3  [18.82786671053566, 18.844866670481338, 18.862...   
27   S1   3  [19.348833358128786, 19.376833474477053, 19.39...   
     S2   3  [19.390599961853013, 19.408600087738023, 19.45...   
     S3   3  [19.48921666094475, 19.505216693369675, 19.517...   
     S4   3  [19.418633346684828, 19.434633379109755, 19.47...   
28   S1   3  [20.67056670099896, 20.702566765848815, 20.744...   
     S2   3  [19.875133398183152, 19.89113343060808, 19.905...   
     S3   3  [19.365666624705, 19.38266658465068, 19.402666...   

                                                   m_start  \
Date S                                                       
17   S1  [142.29960002670282, 288.8056000686645, 298.08...   
     S2  [274.2350000629425, 285.5069998989105, 302.520...   
     S3  [61.913499967575035, 188.4325000209808, 202.27...   
18   S1  [157.60700004959108, 190.90199988746645, 226.9...   
     S2  [36.928783348592106, 44.06178348401386, 93.821...   
     S3  [116.68186676203413, 123.75486671625777, 130.7...   
     S4  [36.65280004768374, 173.73679991989138, 180.74...   
19   S1  [35.68984986839291, 81.69484998283383, 237.851...   
     S2  [36.66781663424172, 47.63181662089028, 54.8928...   
     S3  [36.97419991035463, 72.85619987983705, 117.298...   
     S4  [49.56053342297878, 185.51353349164333, 195.11...   
20   S1  [39.9114168078105, 146.38241676394148, 167.088...   
     S2  [42.41191663424172, 104.07091655413308, 117.22...   
     S3  [42.19556673723855, 307.60756667811074, 409.13...   
     S4  [47.243816655222645, 301.31881670290636, 309.3...   
25   S1  [33.1481666723887, 55.83316661516824, 197.6741...   
     S2  [36.50269981117253, 49.31069989891057, 57.6836...   
     S3  [34.97036653124496, 125.27336650454208, 132.43...   
     S4  [153.80466665712993, 166.89666666475932, 173.9...   
26   S1  [33.8476999904633, 117.56469999809269, 125.488...   
     S2  [32.50146682306922, 69.23946673914588, 80.1964...   
     S3           [34.160233295186345, 185.35323313496906]   
     S4  [120.25486675631203, 129.96286670099892, 136.9...   
27   S1  [62.42183331235242, 164.6368334649403, 172.039...   
     S2  [101.73259996948241, 108.86059999046324, 117.9...   
     S3  [248.27621684023552, 367.37821683832817, 413.4...   
     S4  [42.053633337148085, 409.12563336385097, 450.1...   
28   S1  [42.36056675821942, 53.73556675821942, 164.514...   
     S2  [230.03813350499468, 305.6

In [199]:
# Sample-2 Indexing based on Rat name, Session and Date
df.loc[('RM10','17')]

,dim,time,m_start,m_stop,SF,t_m,lick,r_x,r_y,r_z,...,z_p,x0,y0,z0,moving,RW,r_start,r_stop,NSF,Total Trials
S,,,,,,,,,,,,,,,,,,,,,
S1,3,"[20.343600175476013, 20.411600015258728, 20.43...","[142.29960002670282, 288.8056000686645, 298.08...","[146.30460014114374, 292.61860003242487, 302.6...","[3, 4]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 541.2396333333334, 541.2472666666666, 54...","[3.041215709448993, 3.000524363480997, 5.01705...","[2.9598330175130005, 2.9561338042431826, 2.985...","[2.619505396689759, 2.6158061834199415, 2.0979...",...,531.0 531.0 531.0 531.0 531.0 531.0 531.0 531....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[17369, 38176, 39509, 74341, 75364, 77597]","[17965, 38722, 40151, 74917, 75905, 78152]",2,6
S2,3,"[20.37000005340576, 20.426000047683715, 20.440...","[274.2350000629425, 285.5069998989105, 302.520...","[277.86499993896484, 292.25099985694885, 305.7...","[1, 2, 3, 5, 6]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 279.68016666666665, 279.7112666666667, 2...","[5.005960357380994, 5.01335878392063, 4.758113...","[2.982028297131908, 2.9672314440526364, 2.9709...","[2.2902754156759713, 2.286576202406154, 2.3087...",...,531.0 531.0 531.0 531.0 531.0 531.0 531.0 531....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[36103, 37747, 40256, 41587, 89166, 94160, 960...","[36629, 38757, 40704, 42036, 89654, 94610, 964...",5,8
S3,3,"[19.484500066757164, 19.50050009918209, 19.525...","[61.913499967575035, 188.4325000209808, 202.27...","[65.9195000095367, 191.8205000324249, 205.4205...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 121.46453333333334, 190.52003333333334, ...","[5.01335878392063, 5.01335878392063, 4.7581130...","[2.9968251502111793, 2.9968251502111793, 2.974...","[2.0350297000585402, 2.060924192947265, 2.1608...",...,531.0 531.0 531.0 531.0 531.0 531.0 531.0 531....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[6019, 24021, 25977, 30315, 41982, 44021, 4716...","[6591, 24501, 26416, 30761, 42427, 44474, 4761...",14,16


In [200]:
# Sample-3 Indexing based on Rat name, Session and Date
try_df = df.loc[('RM10','17', 'S2')].to_frame()
try_df.head()

,RM10
,17
,S2
dim,3
time,"[20.37000005340576, 20.426000047683715, 20.440..."
m_start,"[274.2350000629425, 285.5069998989105, 302.520..."
m_stop,"[277.86499993896484, 292.25099985694885, 305.7..."
SF,"[1, 2, 3, 5, 6]"


# Extracting a Given Session (Rat - Date - Session Number)

In [201]:
def get_single_trial(df,date,session,rat):
    new_df = df.loc[(rat, date, session)].to_frame()
    return new_df.T

ddv1 = get_single_trial(df,'18','S3','RM9')
ddv1.head()

,,,dim,time,m_start,m_stop,SF,t_m,lick,r_x,r_y,r_z,...,z_p,x0,y0,z0,moving,RW,r_start,r_stop,NSF,Total Trials
RM9,18,S3,3,"[19.026683361943583, 19.089683564122538, 19.10...","[34.658683569844584, 117.32368353169761, 142.7...","[38.665683539327006, 121.32568338673912, 146.1...","[2, 3, 6, 9, 12, 14, 15, 16, 18, 20, 21, 23]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 122.51423333333334, 122.51763333333334, ...","[4.513964992495222, 4.421484660749774, 4.48437...","[2.7859699938315616, 2.819262913259923, 2.8673...","[2.8044660601806513, 2.8007668469108333, 2.808...",...,361.6 571.1 571.1 361.6 707.5 707.5 571.1 707....,1024,608,531,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0 0 1 0 2 0 3 0 4 ...,"[2191, 14018, 17703, 18863, 21270, 31546, 3251...","[2753, 14597, 18205, 19352, 21850, 32097, 3301...",12,25


In [202]:
# def get_single_trial(df,date,session,rat):
#     rr = df.loc[df['Date'] == date]
#     rr = rr.loc[rr['S'] == session]
#     new_df = rr.loc[rr['rat'] == rat]
#     return new_df
# ddv1=get_single_trial(hdf,'25','S3','RM16')
# ddv1.head()

# Exploring Each Column

In [203]:
# time column

df_time = ddv1['time'].apply(pd.Series) # expanding df.time into its own dataframe
df_time = df_time.rename(columns = lambda x : 'time_' + str(x)) # renaming each variable
df_time # viewing the time dataframe

,,,time_0,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,...,time_93130,time_93131,time_93132,time_93133,time_93134,time_93135,time_93136,time_93137,time_93138,time_93139
RM9,18,S3,19.026683,19.089684,19.105683,19.112684,19.119684,19.128683,19.135684,19.142684,19.150683,19.157683,...,662.368683,662.375683,662.381683,662.389684,662.396683,662.403683,662.410683,662.417683,662.424683,662.431683


In [204]:
# m_start column
df_m_start = ddv1['m_start'].apply(pd.Series)
df_m_start = df_m_start.rename(columns = lambda x : 'm_start_' + str(x))
df_m_start

,,,m_start_0,m_start_1,m_start_2,m_start_3,m_start_4,m_start_5,m_start_6,m_start_7,m_start_8,m_start_9,...,m_start_15,m_start_16,m_start_17,m_start_18,m_start_19,m_start_20,m_start_21,m_start_22,m_start_23,m_start_24
RM9,18,S3,34.658684,117.323684,142.701684,150.789683,167.220683,239.789683,246.797684,254.519683,267.853684,279.617683,...,402.242683,471.920683,496.588683,522.401683,530.446683,544.438684,579.059683,586.521683,598.660683,635.914683


In [205]:
# m_stop column
df_m_stop = ddv1['m_stop'].apply(pd.Series)
df_m_stop = df_m_stop.rename(columns = lambda x : 'm_stop_' + str(x))
df_m_stop

,,,m_stop_0,m_stop_1,m_stop_2,m_stop_3,m_stop_4,m_stop_5,m_stop_6,m_stop_7,m_stop_8,m_stop_9,...,m_stop_15,m_stop_16,m_stop_17,m_stop_18,m_stop_19,m_stop_20,m_stop_21,m_stop_22,m_stop_23,m_stop_24
RM9,18,S3,38.665684,121.325683,146.199683,154.160683,171.227683,243.790684,250.420683,258.522683,271.856684,283.186683,...,406.697683,475.679683,500.593684,525.878683,534.447683,547.885684,583.244684,590.522683,602.662684,642.704683


In [206]:
# SF column
df_SF = ddv1['SF'].apply(pd.Series)
df_SF = df_SF.rename(columns = lambda x : 'SF_' + str(x))
df_SF

,,,SF_0,SF_1,SF_2,SF_3,SF_4,SF_5,SF_6,SF_7,SF_8,SF_9,SF_10,SF_11
RM9,18,S3,2,3,6,9,12,14,15,16,18,20,21,23


In [207]:
# t_m column
df_t_m = ddv1['t_m'].apply(pd.Series)
df_t_m = df_t_m.rename(columns = lambda x : 't_m' + str(x))
df_t_m

,,,t_m0,t_m1,t_m2,t_m3,t_m4,t_m5,t_m6,t_m7,t_m8,t_m9,...,t_m93130,t_m93131,t_m93132,t_m93133,t_m93134,t_m93135,t_m93136,t_m93137,t_m93138,t_m93139
RM9,18,S3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [235]:
# lick column
df_lick = ddv1['lick'].apply(pd.Series)
df_lick = df_lick.rename(columns = lambda x : 'lick_' + str(x))
df_lick = df_lick.astype(int)
df_lick.head()

,,,lick_0,lick_1,lick_2,lick_3,lick_4,lick_5,lick_6,lick_7,lick_8,lick_9,...,lick_947,lick_948,lick_949,lick_950,lick_951,lick_952,lick_953,lick_954,lick_955,lick_956
RM9,18,S3,0,122,122,122,122,122,122,123,123,123,...,648,648,648,648,648,648,648,648,648,651


In [236]:
# Converting lick column to integers and reducing the number of licks
df_lick_int = df_lick.T.drop_duplicates().T
df_lick_int

,,,lick_0,lick_1,lick_7,lick_17,lick_26,lick_36,lick_45,lick_54,lick_60,lick_62,...,lick_872,lick_882,lick_902,lick_914,lick_922,lick_926,lick_936,lick_942,lick_946,lick_956
RM9,18,S3,0,122,123,124,125,126,127,128,129,130,...,639,640,641,642,643,644,645,647,648,651


In [210]:
# r_x column
df_r_x = ddv1['r_x'].apply(pd.Series)
df_r_x = df_r_x.rename(columns = lambda x : 'r_x_' + str(x))
df_r_x

,,,r_x_0,r_x_1,r_x_2,r_x_3,r_x_4,r_x_5,r_x_6,r_x_7,r_x_8,r_x_9,...,r_x_689,r_x_690,r_x_691,r_x_692,r_x_693,r_x_694,r_x_695,r_x_696,r_x_697,r_x_698
RM9,18,S3,4.513965,4.421485,4.484371,4.513965,4.406688,4.451078,5.024456,5.020757,5.028156,5.024456,...,4.776609,4.765511,4.750715,4.735918,4.702625,4.650836,4.617543,4.569453,4.502867,4.451078


In [211]:
# r_y column
df_r_y = ddv1['r_x'].apply(pd.Series)
df_r_y = df_r_y.rename(columns = lambda x : 'r_y_' + str(x))
df_r_y

,,,r_y_0,r_y_1,r_y_2,r_y_3,r_y_4,r_y_5,r_y_6,r_y_7,r_y_8,r_y_9,...,r_y_689,r_y_690,r_y_691,r_y_692,r_y_693,r_y_694,r_y_695,r_y_696,r_y_697,r_y_698
RM9,18,S3,4.513965,4.421485,4.484371,4.513965,4.406688,4.451078,5.024456,5.020757,5.028156,5.024456,...,4.776609,4.765511,4.750715,4.735918,4.702625,4.650836,4.617543,4.569453,4.502867,4.451078


In [212]:
# r_z column
df_r_z = ddv1['r_z'].apply(pd.Series)
df_r_z = df_r_z.rename(columns = lambda x : 'r_z_' + str(x))
df_r_z

,,,r_z_0,r_z_1,r_z_2,r_z_3,r_z_4,r_z_5,r_z_6,r_z_7,r_z_8,r_z_9,...,r_z_689,r_z_690,r_z_691,r_z_692,r_z_693,r_z_694,r_z_695,r_z_696,r_z_697,r_z_698
RM9,18,S3,2.804466,2.800767,2.808165,2.804466,2.804466,2.808165,2.36426,2.36426,2.371658,2.397553,...,2.397553,2.386455,2.416049,2.438244,2.482635,2.575115,2.630603,2.671294,2.719384,2.778572


In [213]:
# Conversion of x_p column to List
x_p_list = ddv1['x_p'].str.split()
x_p_list
# x_p_list_str = x_p_string.split()
# x_p_list_str

RM9  18  S3    [655.4, 916.8, 916.8, 655.4, 655.4, 655.4, 916...
Name: x_p, dtype: object

In [214]:
# x_p column
df_x_p = x_p_list.apply(pd.Series)
df_x_p = df_x_p.rename(columns = lambda x : 'x_p_' + str(x))
df_x_p

,,,x_p_0,x_p_1,x_p_2,x_p_3,x_p_4,x_p_5,x_p_6,x_p_7,x_p_8,x_p_9,...,x_p_90,x_p_91,x_p_92,x_p_93,x_p_94,x_p_95,x_p_96,x_p_97,x_p_98,x_p_99
RM9,18,S3,655.4,916.8,916.8,655.4,655.4,655.4,916.8,655.4,655.4,655.4,...,655.4,614.4,614.4,916.8,916.8,614.4,916.8,655.4,655.4,655.4


In [215]:
# Conversion of y_p column to List
y_p_list = ddv1['y_p'].str.split()
y_p_list

RM9  18  S3    [794.8, 648.8, 648.8, 449.0, 794.8, 794.8, 648...
Name: y_p, dtype: object

In [216]:
# y_p column
df_y_p = y_p_list.apply(pd.Series)
df_y_p = df_y_p.rename(columns = lambda x : 'y_p_' + str(x))
df_y_p

,,,y_p_0,y_p_1,y_p_2,y_p_3,y_p_4,y_p_5,y_p_6,y_p_7,y_p_8,y_p_9,...,y_p_90,y_p_91,y_p_92,y_p_93,y_p_94,y_p_95,y_p_96,y_p_97,y_p_98,y_p_99
RM9,18,S3,794.8,648.8,648.8,449.0,794.8,794.8,648.8,794.8,449.0,449.0,...,794.8,608.0,608.0,568.6,568.6,608.0,648.8,794.8,794.8,449.0


In [217]:
# Conversion of z_p column to List
z_p_list = ddv1['z_p'].str.split()
z_p_list

RM9  18  S3    [361.6, 571.1, 571.1, 361.6, 707.5, 707.5, 571...
Name: z_p, dtype: object

In [218]:
# z_p column
df_z_p = z_p_list.apply(pd.Series)
df_z_p = df_z_p.rename(columns = lambda x : 'z_p_' + str(x))
df_z_p

,,,z_p_0,z_p_1,z_p_2,z_p_3,z_p_4,z_p_5,z_p_6,z_p_7,z_p_8,z_p_9,...,z_p_90,z_p_91,z_p_92,z_p_93,z_p_94,z_p_95,z_p_96,z_p_97,z_p_98,z_p_99
RM9,18,S3,361.6,571.1,571.1,361.6,707.5,707.5,571.1,707.5,361.6,707.5,...,707.5,531.0,531.0,491.3,491.3,531.0,491.3,707.5,707.5,361.6


In [219]:
# x0 column
df_x0 = ddv1['x0']
df_x0

RM9  18  S3    1024
Name: x0, dtype: object

In [220]:
# y0 column
df_y0 = ddv1['y0']
df_y0

RM9  18  S3    608
Name: y0, dtype: object

In [221]:
# z0 column
df_z0 = ddv1['z0']
df_z0

RM9  18  S3    531
Name: z0, dtype: object

In [222]:
# moving column
df_moving = ddv1['moving'].apply(pd.Series)
df_moving = df_moving.rename(columns = lambda x : 'moving' + str(x))
df_moving

,,,moving0,moving1,moving2,moving3,moving4,moving5,moving6,moving7,moving8,moving9,...,moving93130,moving93131,moving93132,moving93133,moving93134,moving93135,moving93136,moving93137,moving93138,moving93139
RM9,18,S3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [223]:
# RW column
RW_list = ddv1['RW'].str.split()
df_RW = RW_list.apply(pd.Series)
df_RW = df_RW.rename(columns = lambda x : 'RW_' + str(x))
# df_RW
ddv1['RW']

RM9  18  S3    0        0
1        0
2        0
3        0
4 ...
Name: RW, dtype: object

In [224]:
# r_start column
df_r_start = ddv1['r_start'].apply(pd.Series)
df_r_start = df_r_start.rename(columns = lambda x : 'r_start' + str(x))
df_r_start

,,,r_start0,r_start1,r_start2,r_start3,r_start4,r_start5,r_start6,r_start7,r_start8,r_start9,...,r_start15,r_start16,r_start17,r_start18,r_start19,r_start20,r_start21,r_start22,r_start23,r_start24
RM9,18,S3,2191,14018,17703,18863,21270,31546,32511,33602,35523,37214,...,54774,64836,68518,72353,73543,75619,80762,81865,83665,89201


In [225]:
# r_stop column
df_r_stop = ddv1['r_stop'].apply(pd.Series)
df_r_stop = df_r_stop.rename(columns = lambda x : 'r_stop' + str(x))
df_r_stop

,,,r_stop0,r_stop1,r_stop2,r_stop3,r_stop4,r_stop5,r_stop6,r_stop7,r_stop8,r_stop9,...,r_stop15,r_stop16,r_stop17,r_stop18,r_stop19,r_stop20,r_stop21,r_stop22,r_stop23,r_stop24
RM9,18,S3,2753,14597,18205,19352,21850,32097,33013,34179,36098,37727,...,55413,65395,69113,72867,74138,76128,81379,82460,84260,90206


In [226]:
# Number of Successful Trials column
numSF = ddv1['NSF'].reset_index().T.reset_index()[0][3:].reset_index()[0][0]
numSF

12

In [41]:
df.head()

dim                                               time  \
rat  Date S                                                           
RM10 17   S1   3  [20.343600175476013, 20.411600015258728, 20.43...   
          S2   3  [20.37000005340576, 20.426000047683715, 20.440...   
          S3   3  [19.484500066757164, 19.50050009918209, 19.525...   
     18   S1   3  [20.1379999485016, 20.193000015258804, 20.2290...   
          S2   3  [21.12278335431415, 21.14078348019916, 21.2137...   

                                                        m_start  \
rat  Date S                                                       
RM10 17   S1  [142.29960002670282, 288.8056000686645, 298.08...   
          S2  [274.2350000629425, 285.5069998989105, 302.520...   
          S3  [61.913499967575035, 188.4325000209808, 202.27...   
     18   S1  [157.60700004959108, 190.90199988746645, 226.9...   
          S2  [36.928783348592106, 44.06178348401386, 93.821...   

                                                         m_stop  \
rat  Date S                                                       
RM10 17   S1  [146.30460014114374, 292.61860003242487, 302.6...   
          S2  [277.86499993896484, 292.25099985694885, 305.7...   
          S3  [65.9195000095367, 191.8205000324249, 205.4205...   
     18   S1  [161.615999874115, 194.55699985885622, 230.535...   
          S2  [40.93078344205219, 48.06778328755695, 97.8277...   

                                                           SF  \
rat  Date S                                                     
RM10 17   S1                                           [3, 4]   
          S2                                  [1, 2, 3, 5, 6]   
          S3  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]   
     18   S1      [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15]   
          S2                   [7, 9, 15, 20, 21, 22, 23, 24]   

                                                            t_m  \
rat  Date S                                                       
RM10 17   S1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
          S2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
          S3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
     18   S1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
          S2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                           lick  \
rat  Date S                                                       
RM10 17   S1  [0.0, 541.2396333333334, 541.2472666666666, 54...   
          S2  [0.0, 279.68016666666665, 279.7112666666667, 2...   
          S3  [0.0, 121.46453333333334, 190.52003333333334, ...   
     18   S1  [0.0, 82.6542, 82.6982, 83.1201, 194.108833333...   
          S2  [0.0, 209.08166666666668, 209.17086666666665, ...   

                                                            r_x  \
rat  Date S                                                       
RM10 17   S1  [3.041215709448993, 3.000524363480997, 5.01705...   
          S2  [5.005960357380994, 5.01335878392063, 4.758113...   
          S3  [5.01335878392063, 5.01335878392063, 4.7581130...   
     18   S1  [5.01335878392063, 4.857991826588281, 4.243922...   
          S2  [1.7608255164332929, 2.834059766339194, 2.7452...   

                                                            r_y  \
rat  Date S                                                       
RM10 17   S1  [2.9598330175130005, 2.9561338042431826, 2.985...   
          S2  [2.982028297131908, 2.9672314440526364, 2.9709...   
          S3  [2.9968251502111793, 2.9968251502111793, 2.974...   
     18   S1  [2.989426723671543, 2.9857275104017256, 2.9931...   
          S2  [2.3975526005006884, 3.21877794640025, 3.41853...   

                                                            r_z  ...  \
rat  Date S                                                      ...   
RM10 17   S1  [2.619505396689759, 2.6158061834199415, 2.0979...  ...   
          S2  [2.290275

# Extracting Trials from a Given Session

In [180]:
trials_frames = pd.DataFrame()
trials_frames['start'] = df_r_start.reset_index().T.reset_index()[0][3:].reset_index()[0]
trials_frames['stop'] = df_r_stop.reset_index().T.reset_index()[0][3:].reset_index()[0]
trials_frames.head()

,start,stop
0,2191,2753
1,14018,14597
2,17703,18205
3,18863,19352
4,21270,21850


In [181]:
# SF column
df_SF = ddv1['SF'].apply(pd.Series)
df_SF = df_SF.rename(columns = lambda x : 'SF_' + str(x))
df_SF

,,,SF_0,SF_1,SF_2,SF_3,SF_4,SF_5,SF_6,SF_7,SF_8,SF_9,SF_10,SF_11
RM9,18,S3,2,3,6,9,12,14,15,16,18,20,21,23


In [182]:
sf = ddv1['SF'].apply(pd.Series).reset_index().T.reset_index()[0][3:].reset_index()[0]
print(type(sf))

<class 'pandas.core.series.Series'>


In [183]:
trials_frames['start'][sf[1]]
numSF

12

In [184]:
sf_trials_frames = pd.DataFrame()
cond = trials_frames.index.isin(sf)
rows = trials_frames.loc[cond, :]
sf_trials_frames = sf_trials_frames.append(rows, ignore_index=True)
trials_frames.drop(rows.index, inplace=True)
trials_frames = trials_frames.reset_index().T[1:].T

In [250]:
sf_trials_frames

,start,stop
0,17703,18205
1,18863,19352
2,32511,33013
3,37214,37727
4,42941,43473
5,53256,53825
6,54774,55413
7,64836,65395
8,72353,72867
9,75619,76128


In [249]:
sf_trials_frames['stop'].iloc[-1]

84260

In [241]:
trials_frames

,start,stop
0,2191,2753
1,14018,14597
2,21270,21850
3,31546,32097
4,33602,34179
5,35523,36098
6,38389,38963
7,40880,41442
8,49677,50254
9,68518,69113


In [263]:
import os
rat = "RM9"
date = 18
session = "S3"

directory_name = "Video Extraction-{rat_}-{date_}-{session_}".format(rat_ = rat, date_ = date, session_ = session)
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

successful_trials = directory_name+"/Successful_Trials"
if not os.path.exists(successful_trials):
    os.makedirs(successful_trials)
    
trials = directory_name+"/Trials"
if not os.path.exists(trials):
    os.makedirs(trials)

# Successful Trials' Frames Extraction

In [304]:
# Successful Trials
cap = cv2.VideoCapture("video.mp4")
if (cap.isOpened() == False):
    print("UNABLE TO OPEN CAMERA FEED.")

currentFrame = 0

i = 0
    
while(True):

    ret, frame = cap.read()
    name = "frame%d.jpg"%currentFrame
    
    if(currentFrame >= sf_trials_frames['start'][i] and currentFrame <= sf_trials_frames['stop'][i]):
        if(currentFrame == sf_trials_frames['start'][i]):
            print ("Storing into Successful_Trial{} Folder...".format(i))
            sub_trials = successful_trials+"/Successful_Trial{}".format(i)
            if not os.path.exists(sub_trials):
                os.makedirs(sub_trials)
                
        cv2.imwrite(sub_trials+"/{}".format(name), frame)

    if(currentFrame == sf_trials_frames['stop'].iloc[-1]):
        break
    if(currentFrame == sf_trials_frames['stop'][i] and currentFrame <= sf_trials_frames['stop'].iloc[-1]):
        print ("Finished Successful Trial{} Folder...".format(i))
        i = i+1
            
    currentFrame = currentFrame+1

cap.release()

Storing into Successful_Trial0 Folder...
Finished Successful Trial0 Folder...
Storing into Successful_Trial1 Folder...
Finished Successful Trial1 Folder...
Storing into Successful_Trial2 Folder...
Finished Successful Trial2 Folder...
Storing into Successful_Trial3 Folder...
Finished Successful Trial3 Folder...
Storing into Successful_Trial4 Folder...
Finished Successful Trial4 Folder...
Storing into Successful_Trial5 Folder...
Finished Successful Trial5 Folder...
Storing into Successful_Trial6 Folder...
Finished Successful Trial6 Folder...
Storing into Successful_Trial7 Folder...
Finished Successful Trial7 Folder...
Storing into Successful_Trial8 Folder...
Finished Successful Trial8 Folder...
Storing into Successful_Trial9 Folder...
Finished Successful Trial9 Folder...
Storing into Successful_Trial10 Folder...
Finished Successful Trial10 Folder...
Storing into Successful_Trial11 Folder...


#  Trials' Frames Extraction

In [283]:
# Trials
# Successful Trials
cap = cv2.VideoCapture("video.mp4")
if (cap.isOpened() == False):
    print("UNABLE TO OPEN CAMERA FEED.")

currentFrame = 0

i = 0
    
while(True):

    ret, frame = cap.read()
    name = "frame%d.jpg"%currentFrame
    
    if(currentFrame >= trials_frames['start'][i] and currentFrame <= trials_frames['stop'][i]):
        if(currentFrame == trials_frames['start'][i]):
            print ("Storing into Trial{} Folder...".format(i))
            sub_trials = trials+"/Trial{}".format(i)
            if not os.path.exists(sub_trials):
                os.makedirs(sub_trials)
                
        cv2.imwrite(sub_trials+"/{}".format(name), frame)

    if(currentFrame == trials_frames['stop'].iloc[-1]):
        break
    if(currentFrame == trials_frames['stop'][i] and currentFrame <= trials_frames['stop'].iloc[-1]):
        print ("Finished Trial{} Folder...".format(i))
        i = i+1
            
    currentFrame = currentFrame+1

cap.release()

Storing into Trial0 Folder...
Finished Trial0 Folder...
Storing into Trial1 Folder...
Finished Trial1 Folder...
Storing into Trial2 Folder...
Finished Trial2 Folder...
Storing into Trial3 Folder...
Finished Trial3 Folder...
Storing into Trial4 Folder...
Finished Trial4 Folder...
Storing into Trial5 Folder...
Finished Trial5 Folder...
Storing into Trial6 Folder...
Finished Trial6 Folder...
Storing into Trial7 Folder...
Finished Trial7 Folder...
Storing into Trial8 Folder...
Finished Trial8 Folder...
Storing into Trial9 Folder...
Finished Trial9 Folder...
Storing into Trial10 Folder...
Finished Trial10 Folder...
Storing into Trial11 Folder...
Finished Trial11 Folder...
Storing into Trial12 Folder...


# Successful Trials' Video Extraction

In [305]:
i = 0
n = len(sf_trials_frames)

while(True):
    print(i)
    img_array = []
    sub_trials = successful_trials+"/Successful_Trial{}".format(i)
    for filename in sorted(glob.glob(sub_trials+"/*.jpg")):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    #print("Checkpoint1- {}".format(i))
    
    out = cv2.VideoWriter(sub_trials+"/SF_video{}.mp4".format(i),cv2.VideoWriter_fourcc(*'DIVX'), 24, size) # 24 is fps(The standard fps value)

    for j in range(len(img_array)):
        out.write(img_array[j])
    out.release()
    
    print ("Video Extraction Complete in Successful_Trial{} Folder.".format(i))
    i = i+1
    if(i == n):
        break

0
Video Extraction Complete in Successful_Trial0 Folder.
1
Video Extraction Complete in Successful_Trial1 Folder.
2
Video Extraction Complete in Successful_Trial2 Folder.
3
Video Extraction Complete in Successful_Trial3 Folder.
4
Video Extraction Complete in Successful_Trial4 Folder.
5
Video Extraction Complete in Successful_Trial5 Folder.
6
Video Extraction Complete in Successful_Trial6 Folder.
7
Video Extraction Complete in Successful_Trial7 Folder.
8
Video Extraction Complete in Successful_Trial8 Folder.
9
Video Extraction Complete in Successful_Trial9 Folder.
10
Video Extraction Complete in Successful_Trial10 Folder.
11
Video Extraction Complete in Successful_Trial11 Folder.


# Trials' Video Extraction

In [303]:
i = 0
n = len(trials_frames)

while(True):
    print(i)
    img_array = []
    sub_trials = trials+"/Trial{}".format(i)
    for filename in sorted(glob.glob(sub_trials+"/*.jpg")):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    #print("Checkpoint1- {}".format(i))
    
    out = cv2.VideoWriter(sub_trials+"/trial_video{}.mp4".format(i),cv2.VideoWriter_fourcc(*'DIVX'), 24, size) # 24 is fps(The standard fps value)

    for j in range(len(img_array)):
        out.write(img_array[j])
    out.release()
    
    print ("Video Extraction Complete in Trial{} Folder.".format(i))
    i = i+1
    if(i == n):
        break

0
Video Extraction Complete in Trial0 Folder.
1
Video Extraction Complete in Trial1 Folder.
2
Video Extraction Complete in Trial2 Folder.
3
Video Extraction Complete in Trial3 Folder.
4
Video Extraction Complete in Trial4 Folder.
5
Video Extraction Complete in Trial5 Folder.
6
Video Extraction Complete in Trial6 Folder.
7
Video Extraction Complete in Trial7 Folder.
8
Video Extraction Complete in Trial8 Folder.
9
Video Extraction Complete in Trial9 Folder.
10
Video Extraction Complete in Trial10 Folder.
11
Video Extraction Complete in Trial11 Folder.
12
Video Extraction Complete in Trial12 Folder.


In [131]:
# df_x_p = df_x_p.transpose()
# df_y_p = df_y_p.transpose()
# df_z_p = df_z_p.transpose()

In [132]:
# df_x_p = df_x_p.reset_index(drop=True)
# df_y_p = df_y_p.reset_index(drop=True)
# df_z_p = df_z_p.reset_index(drop=True)

In [133]:
# df_x_p[71] = df_x_p[71].astype(float)
# df_x_p[71] = df_x_p[71]/1000
# df_y_p[71] = df_y_p[71].astype(float)
# df_y_p[71] = df_y_p[71]/1000
# df_z_p[71] = df_z_p[71].astype(float)
# df_z_p[71] = df_z_p[71]/1000

In [134]:
# x = df_x_p[71]
# y = df_y_p[71]
# z = df_z_p[71]
# handle_h = np.vstack((x, y, z))
# data_handle = [handle_h]

In [135]:
# x

In [136]:
# %matplotlib notebook
# sequence = np.arange(100)
# fig = plt.figure()
# ax = p3.Axes3D(fig)

In [137]:
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('Handle Coordinates')

In [138]:
# ax.set_xlim3d([-1.0, 1.0])
# ax.set_ylim3d([-1.0, 1.0])
# ax.set_zlim3d([-1.0, 1.0])

In [139]:
# lines_handle = [ax.plot(data_handle[0][0,0:100], data_handle[0][1,0:100], data_handle[0][2,0:100], 'o', color = 'c', label = 'Handle')[0]]
# leg = ax.legend();

In [140]:
# ani1 = animation.FuncAnimation(fig, update_lines, 85, fargs=(data_handle, lines_handle),interval=400, blit=False)
# plt.show()